In [1]:
from sklearn import datasets
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

In [4]:
dataset = datasets.load_breast_cancer()
X, y = dataset['data'], dataset['target']
X = torch.FloatTensor(X)
y = torch.FloatTensor(y).view(-1, 1)
X = (X-torch.mean(X)) / torch.std(X)

In [5]:
dset = TensorDataset(X,y)
loader = DataLoader(dset, batch_size=256, shuffle=True)

In [9]:
class NeuralNetwork(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.linear1 = nn.Linear(num_features, 4)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(4,1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, X):
        out = self.linear1(X)
        out = self.relu(out)
        out = self.linear2(out)
        out = self.sigmoid(out)
        return out

In [10]:
model = NeuralNetwork(30)
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [11]:
def train(model, criterion, optimizer, loader):
    epoch_loss = 0
    for X_batch, y_batch in loader:
        optimizer.zero_grad()
        hypothesis = model(X_batch)
        loss = criterion(hypothesis, y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss/len(loader)

In [13]:
n_epochs = 100
for epoch in range(1, n_epochs+1):
    loss = train(model, criterion, optimizer, loader)
    if epoch % 10 == 0:
        print('epoch : {:3d}, loss : {:.4f}'.format(epoch, loss))

epoch :  10, loss : 0.3932
epoch :  20, loss : 0.3412
epoch :  30, loss : 0.2989
epoch :  40, loss : 0.2856
epoch :  50, loss : 0.2544
epoch :  60, loss : 0.2408
epoch :  70, loss : 0.2296
epoch :  80, loss : 0.2337
epoch :  90, loss : 0.2228
epoch : 100, loss : 0.2280


In [14]:
y_predicted = (model(X)>=0.5).float()
score = (y_predicted ==y).float().mean()
print('accuracy : {:.2f}'.format(score))

accuracy : 0.92
